# README.md

# Research question: What’s the initial ADT use time window for each pt? 

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [#Get the numbers of deleted, reviewed and added lines from the chart review](#second-bullet)
* [# Grouped by person Id](#third-bullet)
* [#Iterate, plot windows, save csv](#fourth-bullet)
* [# plot start date; plot inital adt window](#fifth-bullet)

In [ ]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [ ]:
df1=pd.read_excel("../A3test.xls", header=0)

In [ ]:
df1[:5]

In [ ]:
index_list=df1.columns
index_list

In [ ]:
#delete empty columns

print(df1.shape)
del_col=['evidence', 'match_string',
       'start_year', 'start_month', 'start_day', 'end_year', 'end_month',
       'end_day', 'imputed_stage','reason', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19']

for col in del_col:
    del df1[col]
print(df1.shape)

In [ ]:
#change person_id to int
df1['person_id']=df1['person_id'].values.astype(int)

In [ ]:
#df_test=df1[(df1["start_date"] == "may-2014")]
#df_test

In [ ]:
#value=df1.at[483,"start_date"]
#print(value)
#df1.at[483,"start_date"]="2014-05-01 00:00:00"
#value=df1.at[483,"start_date"]
#print(value)

In [ ]:
df1[:5]

In [ ]:
#remove empty cells 
df1['comment'].replace('', np.nan, inplace=True)
df1.dropna(subset=['comment'], inplace=True)
df1['start_date'].replace(u'\xa0', u'', inplace=True)
df1['end_date'].replace(u'\xa0', u'', inplace=True)
df1['start_date'].replace('', np.nan, inplace=True)
df1['end_date'].replace('', np.nan, inplace=True)

In [ ]:
#print(df1.shape)
#df1.to_csv(r'check.csv', index = False)

In [ ]:
#found out the number of pts
df_ptinfo=df1.groupby(["person_id"]).size().reset_index(name='counts')
print(df_ptinfo.shape)

# Get the numbers of deleted, reviewed and added lines from the chart review <a class="anchor" id="second-bullet"></a> 

In [ ]:
deleted_list=[]
reviewed_list=[]
added_list=[]

for ind in df1.index:
    if (df1['comment'][ind]=='deleted'): 
        deleted_list.append(ind)  
    if (df1['comment'][ind]=='reviewed'): 
        reviewed_list.append(ind)  
    if (df1['comment'][ind]=='added'): 
        added_list.append(ind)  

In [ ]:
total_lines=len(deleted_list)+len(reviewed_list)+len(added_list)

print("Total number lines is: " +str(total_lines))
print("Number of deleted lines is: " +str(len(deleted_list)),  "; Percentage is "  +str(100*round(len(deleted_list)/total_lines,3)))
print("Number of reviewed lines is: " +str(len(reviewed_list)), "; Percentage is " +str(100*round(len(reviewed_list)/total_lines,1)))
print("Number of added lines is: " +str(len(added_list)),   "; Percentage is "     +str(100*round(len(added_list)/total_lines,3)))

In [ ]:
###delete the lines that has deleted in the column "comment"
print(df1.shape)
for ind in deleted_list:
    df1.drop(index=ind,inplace=True)
print(df1.shape)

In [ ]:
###Keep only the ones that are ADTs
ADT_lib=["Leuprolide", "Degarelix", "Goserelin", "Triptorelin", "Histrelin"]
not_adt_list=[]

#get the list
for ind in df1.index:
    if (df1['generic_drug_name'][ind] not in ADT_lib): 
        not_adt_list.append(ind) 
        
#delete the lines in the list
for ind in not_adt_list:
    df1.drop(index=ind,inplace=True)
    
print("Number of lines with a non ADT drug is ", len(not_adt_list))
print("shape of cleaned dataframe")
print(df1.shape)
df1.reset_index()

In [ ]:
##check column start_date, end_date time format is correct
#for i in df1.index:
    #print(pd.to_datetime(df1['start_date'][i], format='%d-%b-%Y'))

In [ ]:
###crosscheck
#not_adt_list
###spotcheck 
test2 = df1[(df1["person_id"] == 36062)]
test2

In [ ]:
df1[240:250]

# Group by person_id<a class="anchor" id="third-bullet"></a>

In [ ]:
df2=df1.groupby(["person_id"]).size().reset_index(name='counts')

In [ ]:
df2

In [ ]:
df2.reset_index
df2.shape

In [ ]:
df2.index
df2['index1'] = df2.index

In [ ]:
df2.columns

In [ ]:
df2[['index1','counts']]

In [ ]:
sns.set_style("darkgrid")
sns.set_context("poster", font_scale=1.2, rc={"lines.linewidth": 2})

ax = df2.plot.bar(x='index1', y='counts', width=0.8, color='grey', rot=90, title ="LoT", figsize=(35, 5), legend=True, fontsize=12)

plt.tick_params(axis='x', labelsize=25)
plt.tick_params(axis='y', labelsize=25)
ax.set_xticks(np.arange(0, len(df2.index)+1, 5))
ax.set_xticklabels(np.arange(0, len(df2.index)+1, 5))
plt.xlabel("Person ID", fontsize=30, fontweight="bold")
plt.ylabel("Counts", fontsize=30,fontweight="bold")
plt.legend("")
plt.title("Rows of info per pt",fontsize=30,fontweight="bold")
#ax = df2[['index1'],['counts']].plot(kind='bar', title ="LoT", figsize=(30, 10), legend=True, fontsize=12)

In [ ]:
#get the list of pts with max numbers of entries recording ADT usage
df2["counts"].max()
df2[(df2["counts"] == 6)]

In [ ]:
df2[(df2["counts"] == 6)]

In [ ]:
##check
df1[(df1["person_id"] == 27966)]

In [ ]:
y=27966

df_pt=df1[(df1["person_id"] == y)]
each_pt_drug_start = df_pt['start_date'].to_list()
each_pt_drug_end = df_pt['end_date'].to_list()

In [ ]:
each_pt_drug_start

In [ ]:
##get rid of nan
cleaned_a= [x for x in each_pt_drug_start if str(x) != 'NaN']
cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']

cleaned_b= [x for x in each_pt_drug_end if str(x) != 'NaN']
cleaned_bb= [x for x in cleaned_b if str(x) != 'nan']

##get the date format right
#startdatebin=[datetime.fromisoformat(str(x)) for x in cleaned_aa]
#enddatebin=[datetime.fromisoformat(str(x)) for x in cleaned_bb]

#sort 
#start_sorted_bin=sorted(startdatebin)
#end_sorted_bin=sorted(enddatebin)
start_sorted_bin=sorted(cleaned_aa)
end_sorted_bin=sorted(cleaned_bb)

In [ ]:
start_with_2round=[]
start_with_1round=[]

#for a,b in itertools.combinations(list, 2):
    #print((str(a),str(b)))
a=start_sorted_bin[0]

for b in start_sorted_bin:
#find all adt starting dates after 6months of inital starting    
    if float(abs(a-b).days)>182.5:
        start_with_2round.append(a)
        start_with_2round.append(b)
    else:
        start_with_1round.append(a) 
#cleanup 
start_with_2round=start_with_2round[:2]

#get rid of repeated capturing for one round
cleanlist = []
for i in start_with_1round:
    if i not in cleanlist:
        cleanlist.append(i)
start_with_1round=cleanlist

In [ ]:
if len(start_with_2round)==2:
    print(start_with_2round)
if (len(start_with_2round)<2) and (len(start_with_1round)==1):
    print(start_with_1round)

In [ ]:
enddateall=[]
enddate=[]

for i in end_sorted_bin:
#at least two rounds of ADTs
    if len(start_with_2round)==2:
        if(i>sorted(start_with_2round)[0]) and (i<sorted(start_with_2round)[1]):
            startdate=sorted(start_with_2round)[0]
            enddateall.append(i)
        #pick the one that is most close to the 2nd adt starting date
        enddate=enddateall[-1]
        
#only one round           
    if (len(start_with_2round)<2) and (len(start_with_1round)==1):
        if(i>sorted(start_with_1round)[0]):
            startdate=start_with_1round[0]
            enddateall.append(i)  
        enddate=enddateall[0]

print(y, startdate, enddate, enddate-startdate)

# iterate, plot windows, save csv <a class="anchor" id="fourth-bullet"></a>

In [ ]:
#get the list of pt ids
list_pts=df2["person_id"].to_list()
#list_pts

In [ ]:
#for i in list_pts:
for y in [27966,30455,19566,36062]:    
    #subset the dataframe to each pt
    #get start dates and end dates to lists
    df_pt=df1[(df1["person_id"] == y)]
    
    each_pt_drug_start = df_pt['start_date'].to_list()
    each_pt_drug_end = df_pt['end_date'].to_list()

    ##get rid of nan
    cleaned_a= [x for x in each_pt_drug_start if str(x) != 'NaN']
    cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']

    cleaned_b= [x for x in each_pt_drug_end if str(x) != 'NaN']
    cleaned_bb= [x for x in cleaned_b if str(x) != 'nan']

    ##get the date format right
    #startdatebin=[datetime.fromisoformat(str(x)) for x in cleaned_aa]
    #enddatebin=[datetime.fromisoformat(str(x)) for x in cleaned_bb]

    #sort 
    start_sorted_bin=sorted(cleaned_aa)
    end_sorted_bin=sorted(cleaned_bb)
    
    start_with_2round=[]
    start_with_1round=[]
    a=start_sorted_bin[0]

    for b in start_sorted_bin:
    #find all adt starting dates after 6months of inital starting    
        if float(abs(a-b).days)>182.5:
            start_with_2round.append(a)
            start_with_2round.append(b)
        else:
            start_with_1round.append(a) 
    #cleanup 
    start_with_2round=start_with_2round[:2]

    #get rid of repeated capturing for one round
    cleanlist = []
    for i in start_with_1round:
        if i not in cleanlist:
            cleanlist.append(i)
    start_with_1round=cleanlist
    
    enddateall=[]
    enddate=[]

    for i in end_sorted_bin:
    #at least two rounds of ADTs
        if len(start_with_2round)==2:
            if(i>sorted(start_with_2round)[0]) and (i<sorted(start_with_2round)[1]):
                startdate=sorted(start_with_2round)[0]
                enddateall.append(i)
            #pick the one that is most close to the 2nd adt starting date
            enddate=enddateall[-1]
        
    #only one round           
        if (len(start_with_2round)<2) and (len(start_with_1round)==1):
            if(i>sorted(start_with_1round)[0]):
                startdate=start_with_1round[0]
                enddateall.append(i)  
            enddate=enddateall[0]

    #print(y, startdate, enddate, enddate-startdate)

# plot start date; plot inital adt window <a class="anchor" id="fifth-bullet"></a>

In [ ]:
df_pt=df1[(df1["person_id"]==18)]
df_pt

In [ ]:
result_list=[]
for y in list_pts:    

    #subset the dataframe to each pt
    #get start dates and end dates to lists
    df_pt=df1[(df1["person_id"] == y)]
    
    each_pt_drug_start = df_pt['start_date'].to_list()
    each_pt_drug_end = df_pt['end_date'].to_list()

    ##get rid of nan
    cleaned_a= [x for x in each_pt_drug_start if str(x) != 'NaN']
    cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']

    cleaned_b= [x for x in each_pt_drug_end if str(x) != 'NaN']
    cleaned_bb= [x for x in cleaned_b if str(x) != 'nan']
    
    ##get the date format right
    #startdatebin=[datetime.fromisoformat(str(x)) for x in cleaned_aa]
    #enddatebin=[datetime.fromisoformat(str(x)) for x in cleaned_bb]

    #sort 
    start_sorted_bin=sorted(cleaned_aa)
    end_sorted_bin=sorted(cleaned_bb)
    
    start_with_2round=[]
    start_with_1round=[]
    if(len(start_sorted_bin)>0):
        a=start_sorted_bin[0]
    else:
        startdate='NaN'

    for b in start_sorted_bin:
    #find all adt starting dates after 6months of inital starting    
        if float(abs(a-b).days)>182.5:
            start_with_2round.append(a)
            start_with_2round.append(b)
        else:
            start_with_1round.append(a) 
    #cleanup 
    start_with_2round=start_with_2round[:2]

    #get rid of repeated capturing for one round
    cleanlist = []
    for i in start_with_1round:
        if i not in cleanlist:
            cleanlist.append(i)
    start_with_1round=cleanlist
    
    enddateall=[]

    for i in end_sorted_bin:
    #at least two rounds of ADTs
        if len(start_with_2round)==2:
            if(i>=sorted(start_with_2round)[0]) and (i<=sorted(start_with_2round)[1]):
                startdate=sorted(start_with_2round)[0]
                enddateall.append(i)
            #pick the one that is most close to the 2nd adt starting date
            if len(enddateall) == 0:
                enddate='NaN' 
            else:
                enddate=enddateall[-1]
            
            #when there are end dates that are later than the end of the intial adt usage
            if(i>sorted(start_with_2round)[0]) and (i>sorted(start_with_2round)[1]) and (len(enddateall) == 0):
                startdate=sorted(start_with_2round)[0]
                enddate='NaN'
                #leave it as empty
        
    #only one round           
        if (len(start_with_2round)<2) and (len(start_with_1round)==1):
            if(i>=sorted(start_with_1round)[0]):
                startdate=start_with_1round[0]
                enddateall.append(i)
            if len(enddateall) == 0:
                enddate='NaN' 
            else:
                enddate=enddateall[-1]
    #print(y, startdate, enddate)
    result_per_pt=[y,startdate, enddate]
    result_list.append(result_per_pt)

In [ ]:
df_results=pd.DataFrame(result_list)
df_results.columns=(['person_id', 'inital_ADT_start_date', 'inital_ADT_end_date'])

In [ ]:
df_results

In [ ]:
#create a column for the time window 
df_results['Inital_ADT_timewindow'] = df_results['inital_ADT_end_date'] - df_results['inital_ADT_start_date']

In [ ]:
#create a column for the time window 
df_results['Inital_ADT_timewindow'] = df_results['inital_ADT_end_date'] - df_results['inital_ADT_start_date']
#create a column for the inital adt use window as days
df_results['Inital_ADT_time_window_days']=df_results['Inital_ADT_timewindow'].dt.days

In [ ]:
df_results
df_results['index1'] = df_results.index
df_results

In [ ]:
sns.set_style("darkgrid")
sns.set_context("poster", font_scale=1.2, rc={"lines.linewidth": 2})

ax = df_results.plot.bar(x='index1', y='Inital_ADT_time_window_days', width=0.8, color='pink', rot=90, figsize=(35, 5), legend=True, fontsize=12)

plt.tick_params(axis='x', labelsize=25)
plt.tick_params(axis='y', labelsize=25)

ax.set_xticks(np.arange(0, len(df_results.index)+1, 5))
ax.set_xticklabels(np.arange(0, len(df_results.index)+1, 5))

plt.xlabel("Patient count", fontsize=30, fontweight="bold")
plt.ylabel("Time (days)", fontsize=30,fontweight="bold")
plt.legend("")
plt.title("Inital ADT Use Duration",fontsize=30,fontweight="bold" )
#ax = df2[['index1'],['counts']].plot(kind='bar', title ="LoT", figsize=(30, 10), legend=True, fontsize=12)

In [ ]:
## sns.set_style("darkgrid")
sns.set_context("poster", font_scale=0.8, rc={"lines.linewidth": 2})
binsize=240

bins=np.arange(0, 1500, binsize)
fig, ax = plt.subplots(figsize = (15,5))

#n=df_results['Inital_ADT_time_window_days'].plot(kind = "hist", bins=bins, color="purple", alpha=0.5)
(n, bins, patches) = plt.hist(df_results['Inital_ADT_time_window_days'], bins=bins,color="purple", alpha=0.4)

#decorate
plt.title("Bin Size="+ str(binsize) +" days")
plt.xlabel("Days", fontsize=20, fontweight="bold",color="gray")
plt.ylabel("Population", fontsize=20,fontweight="bold", color="gray")

ax.set_xticks(np.arange(0, 1501, 180))
ax.set_xticklabels(np.arange(0, 1501, 180))

# plot the label/text to each bin
for i in range(0, len(n)):
    x_pos = (bins[i + 1] - bins[i])/4 + bins[i]
    y_pos = n[i] + 0.5
    label = str(int(n[i])) # relative frequency of each bin
    ax.text(x_pos, y_pos, label,fontsize=16,fontweight="bold",color="gray" )
    ax.xaxis.tick_bottom()

In [ ]:
#fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

In [ ]:
bins=np.arange(0, 1500, 90)
bins

In [ ]:
n

In [ ]:
df_results.to_csv("Inital_ADT_use_time_window.csv", encoding='utf-8', index=False)